In [ ]:
import networkx as nx
import os
from pathlib import Path
import pandas as pd
import numpy as np
import math

In [ ]:
xferpath = Path(r"C:\Users\User\Documents\cfb project\data\transferportal\cleaned")
recpath = Path(r"C:\Users\User\Documents\cfb project\data\recruiting\original")
xferfiles = {}
recfiles = {}

for file_path in xferpath.glob('*.graphml'):
    name = file_path.stem
    G_x = nx.read_graphml(file_path)
    xferfiles[name] = G_x

for file_path in recpath.glob('*.graphml'):
    name = file_path.stem
    G_r = nx.read_graphml(file_path)
    recfiles[name] = G_r

print(f"\nSuccessfully loaded {len(xferfiles)} files from xfer and {len(recfiles)} files from rec.")


In [ ]:
# ND + NPV calculation

# This will store all our results
# Format: { 'year': { 'school': ratio } }
all_nd = {}

# Loop through the dictionary of graphs you just loaded
for name, G_x in xferfiles.items():
    
    # Extract the year from the filename "transfer_portal_2024"
    try:
        year = int(name.split('_')[-2])
    except:
        year = name # Fallback if the name is not as expected
        
    print(f"--- Processing {year} ---")
    
    yearly_nd = {}
    yearly_npv = {}
    
    # Loop through every school (node) in this year's graph
    for node in G_x.nodes():
        # Get the raw in-degree and out-degree
        in_deg = G_x.in_degree(node)
        out_deg = G_x.out_degree(node)
        in_weight = in_deg * 
        
        if in_deg != 0 or out_deg != 0:
            nd = in_deg - out_deg
        else:
            nd = 0
        
        yearly_nd[node] = nd
        
    # Add this year's results to the main dictionary
    all_nd[year] = yearly_nd
# --- Convert to a Pandas DataFrame for easy analysis ---
# Rows will be years, Columns will be schools
netdeg_df = pd.DataFrame.from_dict(all_nd, orient='index')

# Sort by year (index) just in case
netdeg_df.sort_index(inplace=True)

print("\n--- Sample of Net Degrees (DataFrame) ---")
# Display the last 5 rows (most recent years)
print(netdeg_df.tail())
print(netdeg_df.loc[2024].sort_values(ascending=False).head(10))

print("\--- Sample of NPVs (DataFrame) ---")
print(npv_df.tail())
print(npv_df.loc[2024].sort_values(ascending=False).head(10))